In [58]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pymongo

In [36]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [46]:
# Mars News URL to scrape
url = 'https://redplanetscience.com/'
browser.visit(url)
html = browser.html

In [48]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [52]:
# Examine the results and look for a ul with the class 'querylist'
result = soup.find('div', id="news",class_='container')

# Retrieve each list item
queries = result.find_all(class_='row')

In [53]:
for query in result:
    print(query)



<div class="col-md-12">
<hr/>
<div class="row">
<div class="col-md-4">
<div class="list_image">
<img src="https://mars.nasa.gov/system/news_items/list_view_images/8506_mars-solar-conjunction-th.jpg"/>
</div>
</div>
<div class="col-md-8">
<div class="list_text">
<div class="list_date">October 6, 2022</div>
<div class="content_title">What's Mars Solar Conjunction, and Why Does It Matter?</div>
<div class="article_teaser_body">NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.</div>
</div>
</div>
</div>
</div>


<div class="col-md-12">
<hr/>
<div class="row">
<div class="col-md-4">
<div class="list_image">
<img src="https://mars.nasa.gov/system/news_items/list_view_images/8630_PIA23768-320x240.jpg"/>
</div>
</div>
<div class="col-md-8">
<div class="list_text">
<div class="list_date">October 6, 2022</div>
<div class="content_title">NASA's Mars Perseverance Rover Gets Its Sample Handling System</div>

In [72]:
# Create empty arrays
titles = []
previews = []

# Loop through returned queries
for query in queries:
    try:
        # Retrieve the query title
        title_search = query.find('div', class_='content_title')

        # Access the query's text content and add to array
        title_text = title_search.text
        titles.append(title_text)
        # Access the query with CSS selectors, get text and add to array
        preview_search = query.find('div', class_='article_teaser_body')
        preview_text = preview_search.text   
        previews.append(preview_text) 

        print('\n-----------------\n')
        print(title_text)
        print(preview_text)

    except Exception as E:
        print(E) 




-----------------

What's Mars Solar Conjunction, and Why Does It Matter?
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.

-----------------

NASA's Mars Perseverance Rover Gets Its Sample Handling System
The system will be collecting and storing Martian rock and soil. Its installation marks another milestone in the march toward the July launch period.

-----------------

NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound
The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet.

-----------------

Mars Helicopter Attached to NASA's Perseverance Rover
The team also fueled the rover's sky crane to get ready for this summer's history-making launch.

-----------------

NASA Mars Mission Connects With Bosnian and Herzegovinian Town
A letter from NASA was presented to the mayor of Jezero, Bos

In [75]:
# Create dictionary to hold articles
article = {titles[i]: previews[i] for i in range(len(titles))}
print(article)

{"What's Mars Solar Conjunction, and Why Does It Matter?": 'NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.', "NASA's Mars Perseverance Rover Gets Its Sample Handling System": 'The system will be collecting and storing Martian rock and soil. Its installation marks another milestone in the march toward the July launch period.', "NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound": "The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet.", "Mars Helicopter Attached to NASA's Perseverance Rover": "The team also fueled the rover's sky crane to get ready for this summer's history-making launch.", 'NASA Mars Mission Connects With Bosnian and Herzegovinian Town': 'A letter from NASA was presented to the mayor of Jezero, Bosnia-Herzegovina, honoring the connection between the town and Jezero Cr